In [0]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from tqdm import tqdm_notebook

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
filename = "drive/Team Drives/Deep Learning Project/game_with_features.csv"
df = pd.read_csv(filename)
df.head(5)

,Unnamed: 0,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,...,Straight_Red_Card,Substitution,Free_Kick_Awarded,Off_Sides,Hand_Ball,Penalty_Awarded,Key_Pass,Failed_Through_Ball,Sending_Off,Own_Goal
0,0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,...,0,0,0,0,0,0,0,0,0,0
1,1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,...,0,0,0,0,0,0,0,0,0,0
2,2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,...,0,0,0,0,0,0,0,0,0,0
3,3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,...,0,0,0,0,0,0,0,0,0,0
4,4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,...,0,0,1,0,0,0,0,0,0,0


In [0]:
df1 = df[['id_odsp','event_team','is_goal', 'assist_method', 'fast_break', 'Shot_Attempted', 'Corner_Kick', 'Foul','First_Yellow_Card', 'Second_Yellow_Card', 'Straight_Red_Card', 'Substitution', 'Free_Kick_Awarded', 'Off_Sides', 'Hand_Ball', 'Penalty_Awarded', 'Key_Pass', 'Failed_Through_Ball', 'Sending_Off', 'Own_Goal',  ]]

In [0]:
# Do aggregation
df1 = df[df['time']<=45].groupby("id_odsp").sum()

# Search for 'home' team
home_team_names = []
away_team_names = []

#for row in tqdm(df1.index):
#  game_id = row
#  home_team_name = df.loc[(df['id_odsp'] == game_id) & (df['side'] == 1)].iloc[0]
#  away_team_name = df.loc[(df['id_odsp'] == game_id) & (df['side'] == 2)].iloc[0]
#  home_team_names.append(home_team_name)
#  away_team_names.append(away_team_name)
  
df1.to_csv("drive/Team Drives/Deep Learning Project/aggregated_game_based_15_features.csv")


In [0]:
df1.head(3)

,Unnamed: 0,sort_order,time,event_type,event_type2,side,shot_place,shot_outcome,is_goal,location,...,Straight_Red_Card,Substitution,Free_Kick_Awarded,Off_Sides,Hand_Ball,Penalty_Awarded,Key_Pass,Failed_Through_Ball,Sending_Off,Own_Goal
id_odsp,,,,,,,,,,,,,,,,,,,,,
004f4ING/,16834268,435,658,122,74.0,41,42.0,12.0,0,65.0,...,0,1,7,2,0,0,0,0,0,0
00LMl81F/,2077918,1431,1186,217,233.0,78,86.0,33.0,3,186.0,...,0,1,11,5,1,0,0,0,0,0
00OX4xFp/,34545498,1378,1193,241,98.0,75,54.0,16.0,0,129.0,...,0,1,16,2,1,0,0,0,0,0


In [0]:
# create the game-based result dictionary since we need to match the result in the ANN.ipynb
import pandas as pd
from  collections import Counter
from copy import copy

df = pd.read_csv("drive/Team Drives/Deep Learning Project/events.csv")
df = df.ix[:,[0,8,9,16]]
game = df['id_odsp'].unique()
game_base = df.groupby('id_odsp')

game_dic = {}
for i in game:
    game_dic[i]=0
    
for each in game:
    this_game = game_base.get_group(each)
    this_game_goal = this_game.groupby('is_goal')


    if len(this_game['is_goal'].unique()) ==2:
        goal = this_game_goal.get_group(1)
        if len(goal['event_team'].unique())==2:
            a = dict(Counter(goal['event_team']))
            temp = copy(a)
            if list(temp.values())[0] > list(temp.values())[1]:
                a['win']=list(temp.keys())[0]
                a['loss']=list(temp.keys())[1]
                a['draw']=None
            else:
                if list(temp.values())[0] == list(temp.values())[1]:
                        a['win']=None
                        a['loss']=None
                        a['draw']=list(temp.keys())
                else:
                    a['win']=list(temp.keys())[1]
                    a['loss']=list(temp.keys())[0]
                    a['draw']=None
            game_dic[each]=a
        else:
            b = dict(Counter(goal['event_team']))
            temp = copy(b)
            other_team = list(this_game['event_team'].unique())
            for i in other_team:
                if list(temp.keys())[0] != i:
                    b[i]=0
                    opponent = i
            b['win'] = list(temp.keys())[0]
            b['loss'] = opponent
            b['draw'] = None
            game_dic[each]=b
    else:
        team = this_game['event_team'].unique()
        c = {}
        for i in team:

            c[i]=0
        temp = copy(c)
        c['win']=None
        c['loss']=None
        c['draw']=list(temp.keys())
        game_dic[each]=c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [0]:
df.columns

Index(['id_odsp', 'event_team', 'opponent', 'is_goal'], dtype='object')

In [0]:
# Hanjing Wang
# have a little modification 
# For each game, We need to aggregate the data for each team.
# For example, for game 'UFotOhit/', we need to count the goals for Borussia and Hamburg seperately. It will make sence

# 36 features
# 18 features for team 0
# 18 features for team 1

filename = "drive/Team Drives/Deep Learning Project/game_with_features.csv"
df = pd.read_csv(filename)
df1 = df[['id_odsp','time', 'event_team','is_goal', 'assist_method', 'fast_break', 'Shot_Attempted', 'Corner_Kick', 
          'Foul','First_Yellow_Card', 'Second_Yellow_Card', 'Straight_Red_Card', 'Substitution', 'Free_Kick_Awarded', 'Off_Sides', 'Hand_Ball', 'Penalty_Awarded', 'Key_Pass', 'Failed_Through_Ball', 'Sending_Off', 'Own_Goal',  ]]
df1 = df1[df1['time']<=80]

is_goal0= []
assist_method0=[]
fast_break0 = []
Shot_Attempted0=[]
Corner_Kick0 = []
Foul0 = []
First_Yellow_Card0 = []
Second_Yellow_Card0=[]
Straight_Red_Card0 = []
Substitution0 = []
Free_Kick_Awarded0 = []
Off_Sides0 = []
Hand_Ball0 = []
Penalty_Awarded0 = []
Key_Pass0 = []
Failed_Through_Ball0=[]
Sending_Off0 = []
Own_Goal0 =[]

is_goal1= []
assist_method1=[]
fast_break1 = []
Shot_Attempted1=[]
Corner_Kick1 = []
Foul1 = []
First_Yellow_Card1 = []
Second_Yellow_Card1=[]
Straight_Red_Card1 = []
Substitution1 = []
Free_Kick_Awarded1 = []
Off_Sides1= []
Hand_Ball1 = []
Penalty_Awarded1 = []
Key_Pass1 = []
Failed_Through_Ball1=[]
Sending_Off1 = []
Own_Goal1 =[]

feature0 = [is_goal0, assist_method0, fast_break0, Shot_Attempted0,Corner_Kick0, Foul0,First_Yellow_Card0, Second_Yellow_Card0, 
            Straight_Red_Card0, Substitution0, Free_Kick_Awarded0, Off_Sides0, Hand_Ball0, Penalty_Awarded0, Key_Pass0, Failed_Through_Ball0, Sending_Off0, Own_Goal0]

feature1 = [is_goal1, assist_method1, fast_break1, Shot_Attempted1,Corner_Kick1, Foul1,First_Yellow_Card1, Second_Yellow_Card1, 
            Straight_Red_Card1, Substitution1, Free_Kick_Awarded1, Off_Sides1, Hand_Ball1, Penalty_Awarded1, Key_Pass1, Failed_Through_Ball1, Sending_Off1, Own_Goal1]

game = list(df1['id_odsp'].unique())

group = df1.groupby('id_odsp')

for game_count in game:

    temp = group.get_group(game_count)
    team = list(temp['event_team'].unique())
    groups = temp.groupby('event_team')
    for j in team:
        temp1 = groups.get_group(j)
        summ = temp1.sum()
        del summ['id_odsp']
        del summ['event_team']
        del summ['time']
        summ = list(summ)
        if j == list(game_dic[game_count].keys())[0]:            
            for i in range(18):
                feature0[i].append(summ[i])
        else:
            for i in range(18):
                feature1[i].append(summ[i]) 
    
            

In [0]:
# generate the 36 features data
df_final = pd.DataFrame({'id_odsp': game, 'is_goal0': is_goal0, 'assist_method0':assist_method0,'fast_break0': fast_break0, 'Shot_Attempted0': Shot_Attempted0,'Corner_Kick0': Corner_Kick0, 
                         'Foul0': Foul0,'First_Yellow_Card0': First_Yellow_Card0, 'Second_Yellow_Card0':Second_Yellow_Card0, 
            'Straight_Red_Card0':Straight_Red_Card0, 'Substitution0':Substitution0, 'Free_Kick_Awarded0': Free_Kick_Awarded0, 'Off_Sides0':Off_Sides0, 'Hand_Ball0':Hand_Ball0, 'Penalty_Awarded0':Penalty_Awarded0, 
                         'Key_Pass0':Key_Pass0, 'Failed_Through_Ball0': Failed_Through_Ball0, 'Sending_Off0':Sending_Off0, 'Own_Goal0':Own_Goal0,'is_goal1': is_goal1, 'assist_method1':assist_method1,'fast_break1': fast_break1, 
                         'Shot_Attempted1': Shot_Attempted1,'Corner_Kick1': Corner_Kick1, 
                         'Foul1': Foul1,'First_Yellow_Card1': First_Yellow_Card1, 'Second_Yellow_Card1':Second_Yellow_Card1, 
            'Straight_Red_Card1':Straight_Red_Card1, 'Substitution1':Substitution1, 'Free_Kick_Awarded1': Free_Kick_Awarded1, 'Off_Sides1':Off_Sides1, 'Hand_Ball1':Hand_Ball1, 'Penalty_Awarded1':Penalty_Awarded1, 
                         'Key_Pass1':Key_Pass1, 'Failed_Through_Ball1': Failed_Through_Ball1, 'Sending_Off1':Sending_Off1, 'Own_Goal1':Own_Goal1})
df_final.to_csv("drive/Team Drives/Deep Learning Project/aggregated_game_based_36_features_80min.csv", index = False)

In [0]:
# print what the data look like
df_final.head(10)

,Corner_Kick0,Corner_Kick1,Failed_Through_Ball0,Failed_Through_Ball1,First_Yellow_Card0,First_Yellow_Card1,Foul0,Foul1,Free_Kick_Awarded0,Free_Kick_Awarded1,...,Straight_Red_Card1,Substitution0,Substitution1,assist_method0,assist_method1,fast_break0,fast_break1,id_odsp,is_goal0,is_goal1
0,7,6,0,0,0,3,10,15,13,10,...,0,2,3,14,6,1,0,UFot0hit/,3,1
1,3,4,0,0,2,2,23,12,13,22,...,0,0,3,8,11,2,0,Aw5DflLH/,2,1
2,6,4,0,0,1,2,10,23,24,12,...,0,2,2,19,6,0,0,bkjpaC6n/,1,0
3,5,4,0,0,1,1,5,18,19,7,...,0,2,2,12,16,0,0,CzPV312a/,1,0
4,2,8,0,0,1,1,8,9,9,8,...,0,1,1,9,11,0,0,GUOdmtII/,1,0
5,3,1,0,0,2,2,21,19,21,21,...,0,2,2,11,0,0,0,lOpzwMkp/,1,0
6,7,8,0,0,2,1,12,8,8,15,...,0,2,2,10,25,0,0,M7PhlM2C/,2,2
7,4,10,0,0,1,1,13,11,11,13,...,0,2,3,13,20,1,0,QuWqjrYa/,2,0
8,12,6,0,0,1,1,13,9,9,13,...,0,2,1,9,13,0,1,UBZQ4smg/,1,3
9,3,7,0,0,3,2,15,11,12,17,...,0,1,1,14,11,0,0,Wn69eU5B/,1,0
